In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [10]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [11]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [12]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130253

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,193848.4650
2018-02-28,193762.8019
2018-03-31,190745.7659
2018-04-30,140895.1654
2018-05-31,140122.2806
2018-06-30,183732.3773
2018-07-31,183671.7297
2018-08-31,192108.2830
2018-09-30,189094.5856


In [13]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    193848.4650
 2018-02-28    193762.8019
 2018-03-31    190745.7659
 2018-04-30    140895.1654
 2018-05-31    140122.2806
 2018-06-30    183732.3773
 2018-07-31    183671.7297
 2018-08-31    192108.2830
 2018-09-30    189094.5856
 2018-10-31    219515.1222
 2018-11-30    242745.5743
 2018-12-31    263955.7616
 2019-01-31    340406.4647
 2019-02-28    277475.1405
 2019-03-31    296547.3561
 2019-04-30    298575.3731
 2019-05-31    256096.4377
 2019-06-30    221597.8924
 2019-07-31    252686.9918
 2019-08-31    260578.1745
 2019-09-30    197704.7760
 2019-10-31    246382.6819
 2019-11-30    252363.6868
 2019-12-31    284956.2947
 2020-01-31    258894.2453
 2020-02-29    249530.1207
 2020-03-31    292294.5189
 2020-04-30    279069.5410
 2020-05-31    340929.5074
 2020-06-30    213280.4355
 2020-07-31    234317.1646
 2020-08-31    239644.5971
 2020-09-30    221006.6948
 2020-10-31    247189.3949
 2020-11-30    266623.2031
 2020-12-31    327959.5062

# ARIMA

In [14]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [15]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.326702
p-value : 0.013726
Critical Values :
	1%: -3.626652
	5%: -2.945951
	10%: -2.611671


In [16]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1030.347, Time=0.09 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1023.564, Time=0.03 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1022.080, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1019.754, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1018.734, Time=0.32 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1019.249, Time=0.21 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1017.002, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1017.305, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1019.031, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1020.183, Time=0.10 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1022.079, Time=0.23 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1015.872, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1017.893, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1020.982, Time=0.14 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1022.074, Time=0.21 sec
 ARIMA(3,0

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [17]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -503.929
Method:                       css-mle   S.D. of innovations          39065.131
Date:                Sat, 11 Sep 2021   AIC                           1015.858
Time:                        04:25:34   BIC                           1022.809
Sample:                             0   HQIC                          1018.406
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.348e+05   1.85e+04     12.689      0.000    1.99e+05    2.71e+05
ar.L1.y        0.4102      0.147      2.793      0.005       0.122       0.698
ar.L2.y        0.2723      0.148      1.837      0.0

# 예측하기

In [18]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([220025.7117164 , 220223.36845012]), array([39065.13110012, 42223.69110795]), array([[143459.46170883, 296591.96172397],
       [137466.4545842 , 302980.28231604]]))
